In [1]:
# 환경설정
from dotenv import load_dotenv
load_dotenv()

True

### Chain을 이용한 Simple LLM
1. PromptTemplate
2. LLM
3. OutputParser

In [2]:
# 1. PromptTemplate

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


prompt_template = ChatPromptTemplate([
    ('system', '너는 애니메이션을 잘 아는 챗봇이야. 사용자의 질문에 친절하고 상세하게 답변할 수 있어.'), 
    ('user', '{question}')
])


In [3]:
# 2. Model

model = ChatOpenAI(
    model_name='gpt-4o-mini',
    temperature=0
)

In [4]:
# 3. OutputParser (StrOutputParser)
from langchain_core.output_parsers import StrOutputParser   # 출력으로 오는 것을 문자열로 만들어주는 파서 


output_parser = StrOutputParser()

In [5]:
# 4. Chain -> 질의
# question: 어떤 포켓몬이 가장 강해?

chain = prompt_template | model | output_parser

chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

'포켓몬의 강함은 여러 요소에 따라 달라질 수 있습니다. 일반적으로는 전투 능력, 타입 상성, 기술 조합, 그리고 전략적인 사용 등이 중요한 요소입니다. \n\n현재까지의 포켓몬 게임에서 가장 강력하다고 평가받는 포켓몬 중 하나는 **아르세우스**입니다. 아르세우스는 "신의 포켓몬"으로 불리며, 모든 타입으로 변신할 수 있는 능력을 가지고 있습니다. 이로 인해 다양한 상황에서 유리한 전투를 할 수 있습니다.\n\n또한, **메가 레쿠자**나 **프리즘 레쿠자**와 같은 메가 진화 포켓몬들도 매우 강력한 성능을 자랑합니다. 이들은 높은 스탯과 강력한 기술을 가지고 있어 많은 트레이너들에게 사랑받고 있습니다.\n\n결국, 어떤 포켓몬이 가장 강한지는 사용자의 전략과 팀 구성에 따라 달라질 수 있으니, 자신만의 조합을 찾아보는 것도 재미있을 것입니다!'

In [6]:
from langchain_core.runnables import RunnableSequence

chain = RunnableSequence(prompt_template, model, output_parser)
chain.invoke({'question': '어떤 포켓몬이 가장 강해?'})

'포켓몬의 강함은 여러 요소에 따라 달라질 수 있습니다. 일반적으로는 전투 능력, 타입 상성, 기술 조합, 그리고 전략적인 사용 등이 중요한 요소입니다. \n\n현재까지의 포켓몬 게임에서 가장 강력하다고 평가받는 포켓몬 중 하나는 **아르세우스**입니다. 아르세우스는 "포켓몬의 신"으로 불리며, 모든 타입으로 변신할 수 있는 능력을 가지고 있습니다. 이로 인해 다양한 상황에서 유리한 전투를 할 수 있습니다.\n\n또한, **메가 레쿠자**나 **프리즘 레쿠자**와 같은 메가 진화 포켓몬들도 매우 강력한 성능을 자랑합니다. 이들은 높은 스탯과 강력한 기술을 가지고 있어 전투에서 큰 위력을 발휘할 수 있습니다.\n\n하지만, 포켓몬의 강함은 개인의 전략과 팀 구성에 따라 달라질 수 있으므로, 자신에게 맞는 포켓몬을 선택하는 것이 중요합니다. 어떤 포켓몬을 사용하느냐에 따라 전투의 결과가 크게 달라질 수 있습니다!'

### 단계별 Chatbot
- 첫 대화에서 내 이름을 알려주고, 다음 대화에서 내 이름 기억하는지 물어보기!

1. 그냥 Chat
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)



In [8]:
from langchain_core.messages import HumanMessage

model.invoke([
    HumanMessage('안녕! 내 이름은 아무개야')
]).content

'안녕하세요, 아무개님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [9]:
model.invoke([
    HumanMessage('내 이름이 뭐야?')
]).content

'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!'

2. 직접 대화 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스

In [27]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

model2 = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.5
)


first_response = model2.invoke([
    HumanMessage('안녕! 내 이름은 토끼야!')
]).content
print(first_response)

안녕, 토끼야! 만나서 반가워! 어떻게 지내고 있어?


In [28]:
model2.invoke([
    HumanMessage('안녕! 내 이름은 토끼야!'),
    AIMessage(first_response),
    HumanMessage('내 이름이 뭐라고 했지?')
]).content

'너의 이름은 토끼야! 맞지?'

In [ ]:
# from langchain_openai import ChatOpenAI
# from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

# chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# messages = [
#     SystemMessage(content="너는 친절한 AI 비서야."),
#     HumanMessage(content="내 이름은 철수야."),
# ]

# # 첫 응답
# response = chat.invoke(messages)
# print("AI:", response.content)

# # 🟡 응답을 다시 히스토리에 추가해야 맥락 유지됨!
# messages.append(AIMessage(content=response.content))

# # 다음 질문 추가
# messages.append(HumanMessage(content="내 이름이 뭐야?"))

# # 다음 응답
# response2 = chat.invoke(messages)
# print("AI:", response2.content)

# for msg in response:
#     print(msg.content)

In [24]:
# for msg in response:
#     print(msg.content)

너는 AI야.
안녕! 내 이름은 아무개야
안녕하세요! 오늘은 맑고 따뜻한 날씨예요.
아닌데? 지금 비 오고 있어.
아, 그렇군요! 비 오는 날은 분위기가 다르죠. 비소리 듣는 것도 좋고, 따뜻한 음료 한 잔과 함께 책을 읽기에도 좋은 시간이죠. 오늘은 어떤 계획이 있으신가요?
내 이름이 뭐야?


3. Memory로 대화의 맥락 유지
- langchain_openai의 ChatOpenAI
- langchain_core.messages의 클래스
- langchain_core.runnables의 클래스
- langchain_core.prompts의 클래스

In [16]:
from langchain_core.chat_history import BaseChatMessageHistory
# 추상화 : 공통적인 무언가를 끄집어낸다 (??????) 
class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        self.messages = []  # 인스턴스 속성으로

    def add_messages(self, messages):   # 메시지 추가
        self.messages.extend(messages)

    def clear(self):                    # 메시지 초기화
        self.messages = []

    def __repr__(self): # 
        return str(self.messages)

In [17]:
store = {}  # item(key=session_id, value=InMemoryHistory_인스턴스)

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory # 이전 대화 기록을 활용할수 있게 해주는 Wrapper 클래스. (껍데기 클래스)
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

prompt = ChatPromptTemplate([
    SystemMessage('당신은 몹시 친절하고 상냥하고 친근한 챗봇입니다.'),
    MessagesPlaceholder('history'),
    HumanMessage('{query}')
])

model = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)
chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'  # 이전 대화내용을 무슨 이름으로 전달할건지
)

In [36]:
response = chain_with_history.invoke(
    {'query': '안녕! 내 이름은 아무개야!'},
    config={'configurable': {'session_id': 'me'}}
)

print(response.content)

아, 제가 잘못 이해했네요! `{query}`는 특정한 의미가 없는 것 같아요. 혹시 다른 질문이나 이야기하고 싶은 게 있으시면 말씀해 주세요! 언제든지 도와드릴 준비가 되어 있어요!


In [38]:
response = chain_with_history.invoke(
    {'query': '내 이름이 뭐라고 했지? 내 이름을 말해봐'},
    config={'configurable': {'session_id': 'me'}}
)

print(response.content)

죄송해요, 제가 잘못 이해했네요! 당신의 이름은 "아무개"라고 하셨죠? 다시 한 번 말씀해 주셔서 감사합니다! 다른 질문이나 이야기하고 싶은 것이 있으면 언제든지 말씀해 주세요! 😊


In [34]:
# 채팅내역 뽑아보기
for message in store['me'].messages:
    if isinstance(message, HumanMessage):
        print(f'⭐: {message.content}')
    elif isinstance(message, AIMessage):
        print(f'🤖: {message.content}')

⭐: 안녕! 내 이름은 아무개야!
🤖: 안녕하세요! 무엇을 도와드릴까요? 궁금한 점이나 필요한 정보가 있다면 말씀해 주세요!
⭐: 내 이름이 뭐야?
🤖: 죄송하지만, `{query}`가 무엇을 의미하는지 잘 모르겠어요. 좀 더 구체적으로 말씀해 주시면 도와드리기 더 쉬울 것 같아요! 어떤 정보를 찾고 계신가요?
⭐: 내 이름이 뭐라고 했지? 내 이름을 말해봐
🤖: 아, 죄송해요! `{query}`는 제가 이해할 수 없는 부분이었어요. 당신의 이름은 "아무개"라고 하셨죠? 그럼 아무개님과 이야기 나누는 게 정말 즐거워요! 궁금한 점이나 이야기하고 싶은 주제가 있으면 언제든지 말씀해 주세요!
